In [1]:
import heapq as hq

class Graph:

    def __init__(self, vertices):
        self.V = vertices
        self.graph = {}
                            
        self.cycles = {}
        self.dijkstras = {}
                           
        
    def dijkstra(self, start):
        distances = {}
        heap = [(0, start)]

        while heap:
            dist, node = hq.heappop(heap)
            if node in distances:
                continue  # Already encountered before
            # We know that this is the first time we encounter node.
            #   As we pull nodes in order of increasing distance, this 
            #   must be the node's shortest distance from the start node.
            distances[node] = dist
            for neighbor, weight in self.graph[node]:
                if neighbor not in distances:
                    hq.heappush(heap, (dist + weight, neighbor))

        return distances
    
    def dijkstraST(self, g, start, trg):
        distances = {}
        heap = [(0, start)]

        while heap:
            dist, node = hq.heappop(heap)
            
            if node == trg:
                return dist
            
            if node in distances:
                continue  # Already encountered before
            # We know that this is the first time we encounter node.
            #   As we pull nodes in order of increasing distance, this 
            #   must be the node's shortest distance from the start node.
            distances[node] = dist
            for neighbor, weight in g[node]:
                if neighbor not in distances:
                    hq.heappush(heap, (dist + weight, neighbor))

        return float('inf')

    
    def AllDijkstra(self):
        for c in self.graph:
            self.dijkstras[c] = g.dijkstra(c)
        return self.dijkstras
    
    
    def leastCycleDijkstra(self, v):
        least_cycle = float('inf')
        for i in range(len(self.graph[v])):
            
            
            label = self.graph[v][i][0]
            weight = self.graph[v][i][1]
            index_delete1 = -1
            index_delete2 = -1
            
            temp_graph = {}
            for i in self.graph:
                temp_graph[i] = self.graph[i].copy()
                
            for i in range(len(temp_graph[v])):
                if temp_graph[v][i][0] == label:
                    index_delete1 = i
                    break
            del temp_graph[v][index_delete1]
            
            for i in range(len(temp_graph[label])):
                if temp_graph[label][i][0] == v:
                    index_delete2 = i
                    break
            del temp_graph[label][index_delete2]        
            
            temp = self.dijkstraST(temp_graph, v, label)
            
            cycle = temp + weight
            if cycle < least_cycle:
                least_cycle = cycle
            del temp_graph
            
            
            
        return least_cycle
                
    def AllCyclesDijkstra(self):
        
        for c in self.graph:
            self.cycles[c] = g.leastCycleDijkstra(c)
        return self.cycles
    
    def compositeDistance(self, x, c):
        return self.dijkstras[x][c] + self.cycles[c]
        
    def compositeDistanceX(self, x):
        least_composite = float('inf')
        for c in range(self.V):
            if self.compositeDistance(x, c) < least_composite:
                least_composite = self.compositeDistance(x, c)
        return least_composite
    
    def girthComposite(self):
        least_composite = float('inf')
        for x in range(self.V):
            cdx = self.compositeDistanceX(x)
            if  cdx < least_composite:
                least_composite = cdx
        return least_composite
    
    def girth(self):
        return min(self.cycles, key=self.cycles.get)
            



graph = {
    "a": [("b", 8), ("c", 100), ("d", 6)],
    "b": [("a", 8), ("e", 5)],
    "c": [("a", 100), ("d", 2)],
    "d": [("a", 6), ("c", 2), ("e", 5), ("g", 3)],
    "e": [("b", 5), ("d", 5), ("f", 5)],
    "f": [("e", 5), ("g", 3), ("h", 6)],
    "g": [("d", 3), ("f", 3), ("h", 4)],
    "h": [("g", 4), ("f", 6)]
}

g = Graph(8)

g.graph = graph

print(g.AllDijkstra())

# print(g.dijkstra(0))
# d(x,y)= g.dijkstra(x)[y]

print(g.AllCyclesDijkstra())

# l(c)= g.leastCycleForEachNode(c)
# g.leastCycleForEachNode(0)
# g.leastCycleDijkstra(2)

  
# d+(x,c) = d(x,c) + l(c)= g.compositeDistance(x, c)
# g.compositeDistance(0, 3)

# d+(x)= min d+(x, c) = g.compositeDistanceX(x)
# g.compositeDistanceX(0)

print(g.girth())



{'a': {'a': 0, 'd': 6, 'b': 8, 'c': 8, 'g': 9, 'e': 11, 'f': 12, 'h': 13}, 'b': {'b': 0, 'e': 5, 'a': 8, 'd': 10, 'f': 10, 'c': 12, 'g': 13, 'h': 16}, 'c': {'c': 0, 'd': 2, 'g': 5, 'e': 7, 'a': 8, 'f': 8, 'h': 9, 'b': 12}, 'd': {'d': 0, 'c': 2, 'g': 3, 'e': 5, 'a': 6, 'f': 6, 'h': 7, 'b': 10}, 'e': {'e': 0, 'b': 5, 'd': 5, 'f': 5, 'c': 7, 'g': 8, 'a': 11, 'h': 11}, 'f': {'f': 0, 'g': 3, 'e': 5, 'd': 6, 'h': 6, 'c': 8, 'b': 10, 'a': 12}, 'g': {'g': 0, 'd': 3, 'f': 3, 'h': 4, 'c': 5, 'e': 8, 'a': 9, 'b': 13}, 'h': {'h': 0, 'g': 4, 'f': 6, 'd': 7, 'c': 9, 'e': 11, 'a': 13, 'b': 16}}
{'a': 24, 'b': 24, 'c': 108, 'd': 16, 'e': 16, 'f': 13, 'g': 13, 'h': 13}
f
